In [2]:
import requests
import json

def crawl_foody_api(page=1):
    url = "https://gappapi.deliverynow.vn/api/delivery/get_shops"
    params = {
        "request_id": "",
        "category_group": "",
        "category_id": "",
        "keyword": "",
        "city_id": 218,  # 218 = Hà Nội
        "district_id": "",
        "sort_type": 3,
        "page": page
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Referer": "https://www.foody.vn/ha-noi",
        "Accept": "application/json"
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print("❌ Error:", e)
        print("🔎 Raw response:", response.text)
        return

    shops = data.get("reply", {}).get("shops", [])
    if not shops:
        print("⚠️ Không tìm thấy địa điểm nào.")
    for shop in shops:
        print({
            "name": shop.get("name"),
            "address": shop.get("address"),
            "rating": shop.get("avg_rating"),
            "price_range": shop.get("price_range"),
            "link": "https://www.foody.vn" + shop.get("url"),
            "image": shop.get("photos")[0].get("value") if shop.get("photos") else ""
        })

crawl_foody_api(page=1)


❌ Error: 403 Client Error: Forbidden for url: https://gappapi.deliverynow.vn/api/delivery/get_shops?request_id=&category_group=&category_id=&keyword=&city_id=218&district_id=&sort_type=3&page=1
🔎 Raw response: {"is_customized":false,"is_login":false,"action_type":2,"error":90309999,"tracking_id":"35372218343-8186-4761-bcc1-f64dfec38f40"}



In [4]:
import requests
import json

def crawl_foody_places(page=1):
    url = "https://www.foody.vn/__get/Place/HomeListPlace"
    params = {
        "t": "1744954230159",
        "page": page,
        "lat": 21.033333,
        "lon": 105.85,
        "count": 12,
        "type": 1
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Referer": "https://www.foody.vn/ha-noi",
        "Accept": "application/json, text/plain, */*",
        "X-Requested-With": "XMLHttpRequest"
    }

    response = requests.get(url, params=params, headers=headers)
    print("Status:", response.status_code)

    try:
        data = response.json()
    except Exception as e:
        print("❌ JSON decode error:", e)
        print("🔎 Raw response (first 500 chars):", response.text[:500])
        return

    places = data.get("Items", [])
    for place in places:
        print({
            "name": place.get("Name"),
            "address": place.get("Address"),
            "rating": place.get("AvgRating"),
            "price_range": f"{place.get('PriceFrom')} - {place.get('PriceTo')}",
            "link": "https://www.foody.vn" + place.get("DetailUrl", ""),
            "image": place.get("PicturePath")
        })

# Test 1 page
crawl_foody_places(1)


Status: 200
{'name': 'Sweetc Hut - Tiệm Bánh Online', 'address': '49 Ngõ 93 Vương Thừa Vũ, P. Khương Trung, Quận Thanh Xuân, Hà Nội', 'rating': 7.928, 'price_range': 'None - None', 'link': 'https://www.foody.vn', 'image': None}
{'name': 'Trang Cháo - Nguyễn Ngọc Vũ', 'address': '125 Nguyễn Ngọc Vũ, P. Yên Hòa, Quận Cầu Giấy, Hà Nội', 'rating': 9.0, 'price_range': 'None - None', 'link': 'https://www.foody.vn', 'image': None}
{'name': 'Trà Sữa Gong Cha - 貢茶 - Hoàng Thành Tower', 'address': 'Hoàng Thành Tower, 114 Mai Hắc Đế, Quận Hai Bà Trưng, Hà Nội', 'rating': 7.564, 'price_range': 'None - None', 'link': 'https://www.foody.vn', 'image': None}
{'name': "Gà Rán & Burger McDonald's - Thái Hà", 'address': 'Tầng Trệt Tòa Nhà Việt Tower, 1 Thái Hà, P. Trung Liệt, Quận Đống Đa, Hà Nội', 'rating': 6.952, 'price_range': 'None - None', 'link': 'https://www.foody.vn', 'image': None}
{'name': 'Gà rán và Mì Ý - Jollibee - Cổ Linh', 'address': 'Tầng 3, AEON Mall Long Biên, 27 Cổ Linh, Quận Long Biên

In [2]:
import os
import json

# Đường dẫn đến thư mục chứa các file JSON
folder_path = "/Users/doa_ai/Developer/Grab-project/ai/crawl_tripadvisor/checkpoints"

# Danh sách để lưu nội dung đã đọc từ từng file JSON
merged_data = []

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                # Nếu mỗi file là 1 list, gộp thẳng vào merged_data
                if isinstance(data, list):
                    merged_data.extend(data)
                else:
                    merged_data.append(data)
            except Exception as e:
                print(f"⚠️ Lỗi đọc file {filename}: {e}")

# Ghi kết quả ra file mới
output_file = "/Users/doa_ai/Developer/Grab-project/ai/crawl_tripadvisor/data_all.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"✅ Đã merge {len(merged_data)} object(s) vào {output_file}")


✅ Đã merge 5 object(s) vào /Users/doa_ai/Developer/Grab-project/ai/crawl_tripadvisor/data_all.json
